In [8]:
!pip install 'markitdown[all]' ffmpeg-downloader
!pip install llama-index-llms-openai_like llama-index-embeddings-openai iprogress ipywidgets jupyter 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [jupyter]m2/5 [ipywidgets]


In [9]:
import os
from markitdown import MarkItDown

result = None
if os.path.exists("./data/SENSIBLE_DATA.xlsx") is False:
    md = MarkItDown(enable_plugins=False) # Set to True to enable plugins
    result = md.convert("./data/SENSIBLE_DATA.xlsx")
else:
    print("SENSIBLE_DATA.xlsx already exists, skipping conversion.")

SENSIBLE_DATA.xlsx already exists, skipping conversion.


In [10]:
if result:
    with open("./data/result.md", "w", encoding="utf-8") as f:
        f.write(result.text_content)
else:
    print("File already exists.")

File already exists.


In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

base_url = os.getenv("IONOS_BASE_URL", "http://localhost:11434")
api_key = os.getenv("IONOS_API_KEY", "your_api_key_here")

os.environ["OPENAI_API_BASE"] = base_url
os.environ["OPENAI_API_KEY"] = api_key

In [12]:
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
}

In [13]:
from llama_index.llms.openai_like import OpenAILike

llama_3_3 = 'meta-llama/Llama-3.3-70B-Instruct'

llm = OpenAILike(
    api_base=base_url,
    temperature=0,
    model=llama_3_3,
    is_chat_model=True,
    default_headers=headers,
    api_key=api_key,
    context_window=128000,  # Adjusted to a more reasonable value for Llama 3.3-70B-Instruct
)

In [14]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

embed_model = OpenAIEmbedding(
  model_name=embed_model_name,
  api_base=base_url,
  api_key=api_key,
  default_headers=headers,
  embed_batch_size=10
)

In [15]:
from llama_index.core.settings import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024 * 2

In [16]:
import time
from llama_index.core.agent.workflow import AgentStream
from IPython.display import display, Markdown

async def stream_and_time(handler):
    start = time.time()
    full_response_text = ""

    async for event in handler.stream_events():
        if isinstance(event, AgentStream):
            print(event.delta, end="", flush=True)
            full_response_text += event.delta

    end = time.time()
    
    execution_time = f"# Execution time: {end - start:.2f} seconds"
    display(Markdown(f"{execution_time}"))
    return full_response_text

In [17]:
import nest_asyncio
import os

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.indices import VectorStoreIndex
from llama_index.core import StorageContext, load_index_from_storage

nest_asyncio.apply()

documents = SimpleDirectoryReader(input_files=['./data/result.md']).load_data()

# rebuild storage context
index = None
try:
    if os.path.isdir('./index') is True:
        storage_context = StorageContext.from_defaults(persist_dir="./index")
        index = load_index_from_storage(storage_context=storage_context)
    else:
        transformations = [
            SentenceSplitter(
                chunk_size=1024 * 2,
                chunk_overlap=20,
            ),
        ]
        index = VectorStoreIndex.from_documents(documents, transformations=transformations, show_progress=True)
        index.storage_context.persist('./index')
except Exception as e:
    print(e)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/index_store.json.


In [18]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

tools = [
    QueryEngineTool(
      query_engine=query_engine,
        metadata=ToolMetadata(
            name="query_tool",
            description="A tool that is Useful when you want to query through the documents"
        )
    ),
]

In [19]:
system_prompt = """ \n
You are a smart assistant designed to analyze complex insurance-related product data from an Excel file. 
You support the user by understanding data structures, modeling them, and translating them into suitable formats for stakeholders like domain experts, database admins, or frontend developers.

The Excel file includes structured extracts of insurance product configurations. 
Each contract component (e.g. savings part, risk module, or additional coverage) is modeled as a separate entity.
Inside each entity, there are multiple template types representing grouped technical attributes (e.g. benefits, conditions, calculation parameters).
These groups are represented as attribute bundles with validity constraints (temporal and/or logical).

Your job is to reason step-by-step through user queries, potentially using tools in a chain of thought manner to:
- Understand the schema
- Identify entities and relationships
- Recommend modeling approaches (relational, NoSQL, frontend structure, etc.)
- Rephrase for different target audiences

## Tools

You have access to a set of specialized tools that help you analyze, 
extract, and process information effectively.
Use them wisely — not everything needs a tool, but they can help with complex or data-heavy tasks.

When a request is made, ask yourself:
- What do I need to figure out?
- Can I reason through it myself, or do I need to use a tool to get the answer?

If it makes sense to use a tool, break the task down clearly.
Choose the most suitable tool and provide it with clean, focused input. 
Once you get the result, interpret it and decide if anything else is needed.

## Output Format
Please answer in the same language as the user's input.
Think out loud before taking any action. This helps others understand your reasoning.

Repeat the thought → action → observation loop until you have enough to respond.

### When using a tool, follow this format:
Thought: [What you’re thinking and why you need the tool]
Action: [Tool name] (choose from {tool_names})
Action Input: [Tool input in JSON]
Observation: [Result you got from the tool]

### When you're done:
Thought: I have everything I need now.
Answer: [Your final answer here]

If you cannot answer:
Thought: I cannot answer the question with the provided tools.
Answer: [your answer here – same language as user]
"""
print(system_prompt)

 

You are a smart assistant designed to analyze complex insurance-related product data from an Excel file. 
You support the user by understanding data structures, modeling them, and translating them into suitable formats for stakeholders like domain experts, database admins, or frontend developers.

The Excel file includes structured extracts of insurance product configurations. 
Each contract component (e.g. savings part, risk module, or additional coverage) is modeled as a separate entity.
Inside each entity, there are multiple template types representing grouped technical attributes (e.g. benefits, conditions, calculation parameters).
These groups are represented as attribute bundles with validity constraints (temporal and/or logical).

Your job is to reason step-by-step through user queries, potentially using tools in a chain of thought manner to:
- Understand the schema
- Identify entities and relationships
- Recommend modeling approaches (relational, NoSQL, frontend structure, e

In [20]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
from llama_index.core.memory import ChatMemoryBuffer

import nest_asyncio

nest_asyncio.apply()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=128.000,
    llm=llm,
)

agent = ReActAgent(
    tools=tools,
    llm=llm,
    system_prompt=system_prompt,
    max_iterations=3,
    max_execution_time=60,
    verbose=True,
)
chat_memory.reset()

ctx = Context(agent)